In [1]:
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po

In [2]:
Time_periods = pe.RangeSet(1, 6)
Generating_units = pe.RangeSet(1, 3)

#Generating units data
c = {1: 5, 2: 15, 3: 30} #Cost of producing
c_U = {1: 800, 2: 500, 3: 250} #Start up cost
P_min = {1: 80, 2: 50, 3: 30} #Output lower limit
P_max = {1: 300, 2: 200, 3: 100} #Output upper limit
S_up = {1: 100, 2: 70, 3: 40} #Maximum starup rate
S_down = {1: 80, 2: 50, 3: 30} #Maximum shutdown rate
R_up = {1: 50, 2: 60, 3: 70} #Maximum ramp-up rate
R_down = {1: 30, 2: 40, 3: 50} #Maximum ramp-down rate
T_up = {1: 3, 2: 2, 3: 1} #Minimum number of time periods required to be on before it can turn off
T_down = {1: 2, 2: 2, 3: 2} #Minimum number of time periods required to be off before it can turn on

#Units operating conditions at t=0
v0 = {1: 1, 2: 0, 3: 0} #On/off status at t=0
p0 = {1: 120, 2: 0, 3: 0} #Output at t=0
U0 = {1: 2, 2: 0, 3: 0} #Number of time periods required to be on at the start of the planning horizon
D0 = {1: 0, 2: 0, 3: 0} #Number of time periods required to be off at the start of the planning horizon

Demand = {1: 240, 2: 250, 3: 200, 4: 170, 5: 230, 6: 190} #Hourly load forecast
Reserve = {1: 10, 2: 10, 3: 10, 4: 10, 5: 10, 6: 10} #Spinning reserve requirement

In [3]:
#Determine solver
solver = po.SolverFactory('cplex')

In [4]:
#Create optmization model
model = pe.ConcreteModel()

In [5]:
#Sets
model.Time_periods = pe.Set(initialize=Time_periods)
model.Generating_units = pe.Set(initialize=Generating_units)

In [6]:
#Constants/Parameters

#Generating units parameters
model.c = pe.Param(model.Generating_units, initialize=c)
model.c_U = pe.Param(model.Generating_units, initialize=c_U)
model.P_min = pe.Param(model.Generating_units, initialize=P_min)
model.P_max = pe.Param(model.Generating_units, initialize=P_max)
model.S_up = pe.Param(model.Generating_units, initialize=S_up)
model.S_down = pe.Param(model.Generating_units, initialize=S_down)
model.R_up = pe.Param(model.Generating_units, initialize=R_up)
model.R_down = pe.Param(model.Generating_units, initialize=R_down)
model.T_up = pe.Param(model.Generating_units, initialize=T_up)
model.T_down = pe.Param(model.Generating_units, initialize=T_down)

#Generating units operation conditions at t=0 parameters
model.v0 = pe.Param(model.Generating_units, initialize=v0)
model.p0 = pe.Param(model.Generating_units, initialize=p0)
model.U0 = pe.Param(model.Generating_units, initialize=U0)
model.D0 = pe.Param(model.Generating_units, initialize=D0)

#Net demands and reserve requirements parameters
model.Demand = pe.Param(model.Time_periods, initialize=Demand)
model.Reserve = pe.Param(model.Time_periods, initialize=Reserve)

In [7]:
#Variables

#Active power produced
model.p = pe.Var(model.Generating_units, model.Time_periods, domain=pe.NonNegativeReals)

#Maximum available power
model.p_max = pe.Var(model.Generating_units, model.Time_periods, domain=pe.NonNegativeReals)

#Binary variables
model.v = pe.Var(model.Generating_units, model.Time_periods, domain=pe.Binary) #On/off status
model.y = pe.Var(model.Generating_units, model.Time_periods, domain=pe.Binary) #Startup status
model.z = pe.Var(model.Generating_units, model.Time_periods, domain=pe.Binary) #Shutdown status

In [8]:
#Objective function (2.1)
expr = sum(model.c[j]*model.p[j,t] + model.c_U[j]* model.y[j,t]
           for j in model.Generating_units for t in Time_periods)
model.objective = pe.Objective(sense=pe.minimize, expr=expr)

In [9]:
#Constraint (2.2): Demand-Generation balancing requirement
model.Constraint22 = pe.ConstraintList()
for t in model.Time_periods:
    lhs = sum(model.p[j,t] for j in model.Generating_units)
    rhs = model.Demand[t]
    model.Constraint22.add(lhs == rhs)

In [10]:
#Constraint (2.3): Total maximum available production capacity = Demand + Reserve
model.Constraint23 = pe.ConstraintList()
for t in model.Time_periods:
    lhs = sum(model.p_max[j,t] for j in model.Generating_units)
    rhs = model.Demand[t] + model.Reserve[t]
    model.Constraint23.add(lhs >= rhs)

In [11]:
#Constraint (2.5): To ensure the logical coherence
model.Constraint25 = pe.ConstraintList()
for j in model.Generating_units:
    for t in model.Time_periods:
        if t-1 <1:
            lhs = model.v0[j] - model.v[j,t] + model.y[j,t] - model.z[j,t]
        if t-1 >= 1:
            lhs = model.v[j,t-1] - model.v[j,t] + model.y[j,t] - model.z[j,t]
        rhs = 0
        model.Constraint25.add(lhs == rhs)

In [12]:
#Constraint (2.7): Ramping up constraint
model.Constraint27 = pe.ConstraintList()
for j in model.Generating_units:
    for t in model.Time_periods:
        if t-1 <1:
            lhs = model.p[j,t] - model.p0[j]
            rhs = model.R_up[j]*model.v0[j] + model.S_up[j]*model.y[j,t]
        if t-1 >= 1:
            lhs = model.p[j,t] - model.p[j,t-1]
            rhs = model.R_up[j]*model.v[j,t-1] + model.S_up[j]*model.y[j,t]
        model.Constraint27.add(lhs <= rhs)

In [13]:
#Constraint (2.8): Ramping down constraint
model.Constraint28 = pe.ConstraintList()
for j in model.Generating_units:
    for t in model.Time_periods:
        if t-1 <1:
            lhs = model.p0[j] - model.p[j,t]
        if t-1 >= 1:
            lhs = model.p[j,t-1] - model.p[j,t]
        rhs = model.R_down[j]*model.v[j,t] + model.S_down[j]*model.z[j,t]
        model.Constraint28.add(lhs <= rhs)

In [14]:
#Constraint (2.9): Uptime constraint
model.Constraint29 = pe.ConstraintList()
for j in model.Generating_units:
    for t in range(min(len(Time_periods),model.U0[j])+1, len(Time_periods)):
        lhs = sum(model.y[j,k]
                  for k in range(max(1,t-model.T_up[j])+1,t))
        rhs = model.v[j,t]
    model.Constraint29.add(lhs <= rhs)

In [15]:
#Constraint (2.10): Downtime constraint
model.Constraint210 = pe.ConstraintList()
for j in model.Generating_units:
    for t in range(min(len(Time_periods),model.D0[j])+1, len(Time_periods)):
        lhs = model.v[j,t] + sum(model.z[j,k]
                  for k in range(max(1,t-model.T_down[j])+1,t))
        rhs = 1
    model.Constraint210.add(lhs <= rhs)

In [16]:
#Constraint (2.11a): Generation Limits
model.Constraint211a = pe.ConstraintList()
for j in model.Generating_units:
    for t in model.Time_periods:
        lhs = model.P_min[j]*model.v[j,t]
        rhs = model.p[j,t]
        model.Constraint211a.add(lhs <= rhs)

In [17]:
#Constraint (2.11b): Generation Limits
model.Constraint211b = pe.ConstraintList()
for j in model.Generating_units:
    for t in model.Time_periods:
        lhs = model.p[j,t]
        rhs = model.p_max[j,t]
        model.Constraint211b.add(lhs <= rhs)

In [18]:
#Constraint (2.11c): Generation Limits
model.Constraint211c = pe.ConstraintList()
for j in model.Generating_units:
    for t in model.Time_periods:
        lhs = model.p_max[j,t]
        rhs = model.P_max[j]*model.v[j,t]
        model.Constraint211c.add(lhs <= rhs)

In [19]:
#Constraint (2.12): Generation Limits
model.Constraint212 = pe.ConstraintList()
for j in model.Generating_units:
    for t in model.Time_periods:
        lhs = model.p_max[j,t]
        if t-1 < 1:
            rhs = model.p0[j] + model.R_up[j]*model.v0[j] + model.S_up[j]*model.y[j,t]
        if t-1 >= 1:
            rhs = model.p[j,t-1] + model.R_up[j]*model.v[j,t-1] + model.S_up[j]*model.y[j,t]
        model.Constraint212.add(lhs <= rhs)

In [20]:
#Constraint (2.13): Generation Limits
model.Constraint213 = pe.ConstraintList()
for j in model.Generating_units:
    for t in range(1, len(Time_periods)-1):
        lhs = model.p_max[j,t]
        rhs = model.P_max[j]*(model.v[j,t]-model.z[j,t+1]) + model.z[j,t+1]*model.S_down[j]
        model.Constraint213.add(lhs <= rhs)

In [21]:
results = solver.solve(model, tee=True)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\G33590~1\AppData\Local\Temp\tmpmlg8lrd3.cplex.log' open.
CPLEX> Problem 'C:\Users\G33590~1\AppData\Local\Temp\tmppd36_vfb.pyomo.lp' read.
Read time = 0.00 sec. (0.01 ticks)
CPLEX> Problem name         : C:\Users\G33590~1\AppData\Local\Temp\tmppd36_vfb.pyomo.lp
Objective sense      : Minimize
Variables            :      91  [Nneg: 37,  Binary: 54]
Objective nonzeros   :      36
Linear constraints   :     157  [Less: 125,  Greater: 7,  Equal: 25]
  Nonzeros           :     463
  RHS nonzeros       :      20

Variables            : Min LB: 0.0000000        Max UB: 1.000000       
Objecti